In [6]:
#Imports
import cv2
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import logging
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor
import multiprocessing
import job
from image_segmentation import segmentate_images as seg

covid_path = os.path.join('dataset/covid')
covid_masks_path = os.path.join('cov_masks')

non_covid_path = os.path.join('dataset/normal')
non_covid_masks_path = os.path.join('non_cov_masks')

In [ ]:
#Read images
def readImages(path):
    return [cv2.imread(file, cv2.IMREAD_GRAYSCALE) for file in glob.glob(path + "/*g")]

with ThreadPoolExecutor() as executor:
    covid_images = executor.submit(readImages, covid_path)
    covid_masks = executor.submit(readImages, covid_masks_path)

    non_covid_images = executor.submit(readImages, non_covid_path)
    non_covid_masks = executor.submit(readImages, non_covid_masks_path)

In [3]:
#Processing
def process(images, masks):
    list = []
    with multiprocessing.Pool() as pool:
        list.append(pool.map(job.job, np.swapaxes([images, masks], 0, 1)))
    return np.squeeze(np.asarray(list))

with ThreadPoolExecutor() as executor:
    cov_processed = executor.submit(process, covid_images.result(), covid_masks.result())
    non_cov_processed = executor.submit(process, non_covid_images.result(), non_covid_masks.result())

In [103]:
#Saving
def save_images(images, save_path):
    for i in range(0, len(images)):
        cv2.imwrite(save_path + '/img' + str(i) + '.png', images[i])

cov_save_path = os.path.join('cov_processed')
non_cov_save_path = os.path.join('non_cov_processed')

save_images(cov_processed.result(), cov_save_path)
save_images(non_cov_processed.result(), non_cov_save_path)


In [102]:
#Generating histogram
def histogram(images, save_path):
    for i in range(0, len(images)):
        plt.figure()
        histg = cv2.calcHist([images[i]],[0],None,[254],[1,255])   #calculating histogram
        plt.plot(histg)
        plt.savefig(save_path + '/img' + str(i) + '.png')
        plt.close()

histogram(non_cov_processed.result(), os.path.join('non_cov_histograms'))
histogram(cov_processed.result(), os.path.join('cov_histograms'))

In [8]:
#Segmentation
seg(covid_path, 'cov_masks')
seg(non_covid_path, 'non_cov_masks')

28/28 [==============================] - 1s 44ms/step


In [ ]:
#Showing
def showImg(img):
    plt.plot(cdf_normalized, color = 'b')
    plt.hist(img.flatten(),256,[0,256], color = 'r')
    plt.xlim([0,256])
    plt.legend(('cdf','histogram'), loc = 'upper left')
    plt.show()